In [1]:
# Parameters
date = "2022-02"


In [2]:
import pandas as pd

from snowflake.connector.pandas_tools import pd_writer
import snowflake.connector.pandas_tools
import snowflake.connector
from datetime import datetime
from pathlib import Path, PurePosixPath

import os
from dotenv import load_dotenv
# Load env variable (to get credentials)
load_dotenv(dotenv_path=Path().absolute().as_posix().split('working_files')[0]  + '/working_files/dags/.env')

True

In [3]:
# Read AEON Subang data from parquet file which was saved previously
data_path = Path().absolute().as_posix().split('working_files')[0]  + '/working_files/data'

df_raw = pd.read_parquet(data_path + '/price_catcher_raw/aeon.parquet')
df_raw.head()

,date,premise_code,item_code,price,premise,address,premise_type,state,district,item,unit,item_group,item_category,etl_time
10351,2022-02-03,3178,1,9.05,AEON BIG SUBANG JAYA,"NO 3 JALAN SS16/1,47500 SUBANG JAYA, SELANGOR ...",Hypermarket,Selangor,Petaling,AYAM BERSIH - STANDARD,1kg,BARANGAN SEGAR,AYAM,2024-05-29T19:52:48.446391
10352,2022-02-03,3178,16,4.70,AEON BIG SUBANG JAYA,"NO 3 JALAN SS16/1,47500 SUBANG JAYA, SELANGOR ...",Hypermarket,Selangor,Petaling,BETIK BIASA,1kg,BARANGAN SEGAR,BUAH-BUAHAN,2024-05-29T19:52:48.446391
10353,2022-02-03,3178,18,5.90,AEON BIG SUBANG JAYA,"NO 3 JALAN SS16/1,47500 SUBANG JAYA, SELANGOR ...",Hypermarket,Selangor,Petaling,PISANG BERANGAN,1kg,BARANGAN SEGAR,BUAH-BUAHAN,2024-05-29T19:52:48.446391
10354,2022-02-03,3178,19,6.50,AEON BIG SUBANG JAYA,"NO 3 JALAN SS16/1,47500 SUBANG JAYA, SELANGOR ...",Hypermarket,Selangor,Petaling,PISANG EMAS,1kg,BARANGAN SEGAR,BUAH-BUAHAN,2024-05-29T19:52:48.446391
10355,2022-02-03,3178,21,4.20,AEON BIG SUBANG JAYA,"NO 3 JALAN SS16/1,47500 SUBANG JAYA, SELANGOR ...",Hypermarket,Selangor,Petaling,TEMBIKAI MERAH TANPA BIJI,1kg,BARANGAN SEGAR,BUAH-BUAHAN,2024-05-29T19:52:48.446391


In [4]:
# Connect to Snowflake via its API
ctx = snowflake.connector.connect(  user=       os.environ['USER'],
                                    password=   os.environ['PASSWORD'],
                                    account=    os.environ['ACCOUNT'],
                                    warehouse=  os.environ['WH'],
                                    database=   os.environ['DB'],
                                    schema=     os.environ['SCHEMA'],
                                    role=       os.environ['ROLE'])

# Get maximum date of data loaded so that we only import new data later
try:
    a = ctx.cursor().execute("SELECT MAX(CAST(DATE/1000000 as timestamp)) as date FROM PRICE_CATCHER.raw.aeon")
    # '
except:
    print("Table doesnt exist, creating....")
    # Automatically create a DB & Schema if it doesnt exist yet
    ctx.cursor().execute("CREATE DATABASE PRICE_CATCHER")
    ctx.cursor().execute("CREATE SCHEMA RAW")
    
    # Automatically create a table if it doesnt exist yet
    snowflake.connector.pandas_tools.write_pandas(
                                            conn=ctx,
                                            df = df_raw,
                                            table_name='AEON',
                                            database='PRICE_CATCHER',
                                            schema='RAW', quote_identifiers=False,
                                            auto_create_table = True
                                            )
    a = ctx.cursor().execute("SELECT MAX(CAST(DATE/1000000 as timestamp)) as date FROM PRICE_CATCHER.raw.aeon")


In [5]:
df = a.fetch_pandas_all()
df.head()

,DATE
0,2022-02-28


In [6]:
try:
    date_existing = df.DATE.dt.date.values[0].isoformat()
except:
    date_existing = '2020-01-01' # Set default value of the above contains nothing
date_existing

'2022-02-28'

In [7]:
# Print the maximum date in the AEON Subang data
new_date = df_raw['date'].max().date().isoformat()
new_date

'2022-02-28'

In [8]:
# Filter rows that is not yet exist in Snowflake
df_to_load = df_raw[df_raw['date'] > date_existing]
print(len(df_to_load))

0


In [9]:
# Write the filtered rows into Snowflake using the same connection credentials previously
snowflake.connector.pandas_tools.write_pandas(
    conn=ctx,
    df = df_to_load,
    table_name='AEON',
    database='PRICE_CATCHER',
    schema='RAW', quote_identifiers=False
)

/tmp/ipykernel_25571/2829828559.py:2: UserWarning: Pandas Dataframe has non-standard index of type <class 'pandas.core.indexes.base.Index'> which will not be written. Consider changing the index to pd.RangeIndex(start=0,...,step=1) or call reset_index() to keep index as column(s)
  snowflake.connector.pandas_tools.write_pandas(


(True,
 1,
 0,
 [('garcvktwqf/file0.txt', 'LOADED', 0, 0, 1, 0, None, None, None, None)])